In [5]:
import json
from collections import defaultdict
import pandas as pd
import os

""" 시공도 작성 """

folder_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\급경사\시공도\vds 데이터\이상치 제거\테스트"
save_path = os.path.join(folder_path, "pivot")
os.makedirs(save_path, exist_ok=True)
json_path = r"C:\Workspace\vissimTest_Python\network_version2\other\construction\route_name.json"

csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".csv")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

# 노선명 json 파일 읽어옴
with open(json_path, "r", encoding="utf-8") as f:
    route_name = json.load(f)

result_list = []
for file_name in csv_files:
    print(file_name + "작업중")
    df = pd.read_csv(os.path.join(folder_path, file_name), encoding="euc-kr")
    # vds_id에서 vds 추출
    df["vds"] = df["vds_id"].str.extract(r'(\d{4})')
    result = df.groupby(["vds_id", "point", "hour", "vds"]).mean().reset_index()
    result_list.append(result)
final_df = pd.concat(result_list, ignore_index=True)
final_df = final_df.groupby(["vds_id", "point", "hour", "vds"]).mean().reset_index()

# 삭제할 컬럼 제거
for col in ["lane_number", "what"]:
    if col in final_df.columns:
        final_df = final_df.drop(col, axis=1)

final_df["type"] = final_df["vds_id"].str.extract(r'\d{4}(VD.)')


# vds, vde 구분 및 그룹화
vds_type_groups = {v: df for v, df in final_df.groupby(["vds", "type"])}

# 딕셔너리 자동 생성
vds_grouped = defaultdict(dict)

# 2차원 배열이 아니라, vds_val이라는 딕셔너리 안에 type_val 이라는 딕셔너리를 추가하는 것!
"""
vds_grouped = {
    '0010' : {
        'VDE' : df1
    }
}
"""
for (vds_val, type_val), df in vds_type_groups.items():
    vds_grouped[vds_val][type_val] = df

for vds_val, type_dict in vds_grouped.items():

    route = route_name.get(vds_val, "UnknownRoute")

    excel_file = os.path.join(save_path, f"{route}.xlsx")

    with pd.ExcelWriter(excel_file, engine="openpyxl") as writer:

        for type_val, df in type_dict.items():

            pivot_df = df.pivot_table(
                index="point",
                columns="hour",
                values="avgspeed",
                aggfunc="mean"
            )

            pivot_df.to_excel(writer, sheet_name=f"{type_val}")

    print(f"{excel_file} 저장 완료!")
print("=== 모든 파일 저장 완료 ===")



data_2020_11_01.csv작업중
data_2020_11_07.csv작업중
data_2020_11_08.csv작업중
data_2020_11_14.csv작업중
data_2020_11_15.csv작업중
data_2020_11_21.csv작업중
data_2020_11_22.csv작업중
data_2020_11_28.csv작업중
data_2020_11_29.csv작업중
data_2021_11_06.csv작업중
data_2021_11_07.csv작업중
data_2021_11_13.csv작업중
data_2021_11_14.csv작업중
data_2021_11_20.csv작업중
data_2021_11_21.csv작업중
data_2021_11_27.csv작업중
data_2021_11_28.csv작업중
data_2022_11_05.csv작업중
data_2022_11_06.csv작업중
data_2022_11_12.csv작업중
data_2022_11_13.csv작업중
data_2022_11_19.csv작업중
data_2022_11_20.csv작업중
data_2022_11_26.csv작업중
data_2022_11_27.csv작업중
VDS_차로_1시간(20231104).csv작업중
VDS_차로_1시간(20231105).csv작업중
VDS_차로_1시간(20231111).csv작업중
VDS_차로_1시간(20231112).csv작업중
VDS_차로_1시간(20231118).csv작업중
VDS_차로_1시간(20231119).csv작업중
VDS_차로_1시간(20231125).csv작업중
VDS_차로_1시간(20231126).csv작업중
C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\급경사\시공도\vds 데이터\이상치 제거\테스트\pivot\경부선.xlsx 저장 완료!
C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\급경사\시공도\vds 데이터\이상치 제거\테스트\pivot\남해선(순천-부산).xlsx 저장 완료!
C:\Users\(주)내일이엔시 도로교통안전